# Pickups and Dropoffs for 10 NYC Hotels (1 Month) (Cont'd)

I'll take the output from the first version of this notebook and use it to draw heat maps of both pickup and dropoff locations for the 10 hotels over the 1 month used in the pilots data.

In [1]:
# imports...
import csv, imp, os, numpy as np, pandas as pd, matplotlib.pyplot as plt
import gmplot
from IPython.display import Image, display
from IPython.core.display import HTML
import webbrowser

# importing helper methods
from util import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# matplotlib setup
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## Maps for Nearby Pick-ups

### Importing Data

Let's grab the (quite large) file "Nearby Pickups and Dropoffs.xlsx" and store the "Nearby Pick-ups" trips' drop off locations in main memory, from which we'll draw heat maps.

In [2]:
# load up the workbook and worksheet for working with trips that have nearby pick-up locations
nearby_pickups = pd.read_excel('../data/Nearby Pickups and Dropoffs.xlsx', sheetname='Nearby Pick-ups')

print nearby_pickups['Hotel Name'].unique()

nearby_pickup_coords = {}
for hotel_name in nearby_pickups['Hotel Name'].unique():
    # get the latitude, longitude coordinates of the corresponding pick-up locations for the trips
    nearby_pickup_coords[hotel_name] = np.array(zip(nearby_pickups.loc[nearby_pickups['Hotel Name'] == hotel_name]['Latitude'], nearby_pickups.loc[nearby_pickups['Hotel Name'] == hotel_name]['Longitude'])).T

# delete the 'nearby_pickups' dataframe from memory
del nearby_pickups

[u'Hilton Millenium Hotel' u'Marriott New York Downtown'
 u'Grand Hyatt New York' u'Waldorf Astoria New York' u'The Roosevelt Hotel'
 u'Hotel Pennsylvania' u'Wyndham New Yorker Hotel'
 u'Marriott New York Marquis' u'Hilton New York Midtown'
 u'Sheraton Hotel New York Times Square']


In [3]:
for hotel_name in nearby_pickup_coords:
    print 'Number of nearby pick-up taxicab trips (within 300 feet of hotel ' + hotel_name + '): ' + str(len(nearby_pickup_coords[hotel_name][0]))

Number of nearby pick-up taxicab trips (within 300 feet of hotel Waldorf Astoria New York): 23923
Number of nearby pick-up taxicab trips (within 300 feet of hotel Wyndham New Yorker Hotel): 21492
Number of nearby pick-up taxicab trips (within 300 feet of hotel The Roosevelt Hotel): 18850
Number of nearby pick-up taxicab trips (within 300 feet of hotel Hilton Millenium Hotel): 7817
Number of nearby pick-up taxicab trips (within 300 feet of hotel Marriott New York Marquis): 23474
Number of nearby pick-up taxicab trips (within 300 feet of hotel Hotel Pennsylvania): 112036
Number of nearby pick-up taxicab trips (within 300 feet of hotel Hilton New York Midtown): 26898
Number of nearby pick-up taxicab trips (within 300 feet of hotel Marriott New York Downtown): 7573
Number of nearby pick-up taxicab trips (within 300 feet of hotel Sheraton Hotel New York Times Square): 19863
Number of nearby pick-up taxicab trips (within 300 feet of hotel Grand Hyatt New York): 36840


### Drawing Maps of Corresponding Drop-off Locations

Let's use the Google Maps plotting Python package (gmplot)[https://pypi.python.org/pypi/gmplot/1.0.5] to overlay a heatmap over a map of NYC (as in (NYC Taxicab Data Experiment)[https://github.com/djsaunde/NYCHotelData/blob/master/code/NYC%20Taxicab%20Data%20Experiment.ipynb]).

In [5]:
for hotel_name in nearby_pickup_coords:
    # some heat map parameters
    map_name = hotel_name + '_Jan2016_300ft_nearby_pickups_destinations_markers.html'
    filepath = '../img/' + map_name[:-5] + '.png'

    gmap = gmplot.GoogleMapPlotter(np.mean(nearby_pickup_coords[hotel_name][0]), np.mean(nearby_pickup_coords[hotel_name][1]), 13)

    # plot the map
    gmap.scatter(nearby_pickup_coords[hotel_name][0], nearby_pickup_coords[hotel_name][1])

    # draw the map
    gmap.draw('../img/' + map_name)

    # display it in the web browser
    webbrowser.open('../img/' + map_name)

## Heat Maps for Nearby Drop-offs

### Importing Data

Let's do the same as above, but with the "Nearby Drop-offs" sheet.

In [30]:
# load up the workbook and worksheet for working with trips that have nearby drop-off locations
nearby_dropoffs = pd.read_excel('../data/Nearby Pickups and Dropoffs.xlsx', sheetname='Nearby Drop-offs')

# get the latitude, longitude coordinates of the corresponding pick-up locations for the trips
nearby_dropoff_coords.append(np.array(zip(nearby_dropoffs['Latitude'], nearby_dropoffs['Longitude'])).T)

# delete the 'nearby_dropoffs' dataframe from memory
del nearby_dropoffs

In [46]:
print 'Number of nearby drop-off taxicab trips (within 300 feet of hotel):', len(nearby_dropoff_coords[0])

Number of nearby drop-off taxicab trips (within 300 feet of hotel): 267835


### Drawing the Heat Map of Corresponding Pick-up Locations

Same as above, but for corresponding pick-ups locations.

In [37]:
# some heat map parameters
map_name = '10Hotels_Jan2016_300ft_nearby_dropoff_starting_points_heatmap.html'
filepath = '../img/' + map_name[:-5] + '.png'

gmap = gmplot.GoogleMapPlotter(np.mean(nearby_dropoff_coords[0]), np.mean(nearby_dropoff_coords[1]), 13)

# plot the heat map
gmap.heatmap(nearby_dropoff_coords[0], nearby_dropoff_coords[1], radius=50, opacity=0.9)

# draw the heat map
gmap.draw('../img/' + map_name)

# display it in the web browser
webbrowser.open('../img/' + map_name)

True

# Reducing Distance Away from Hotel

The heat map so far assigns very low density to most points on the map of NYC. My intuition is that the radius about the hotel for which we deem rides to be "close" (in the sense that we count them as starting (ending) from (at) the hotel) is too large. So, a reduction of the radius should make the heat map more clear.

In [13]:
# new distance criterion in feet
new_distance = 100

## Reducing Distance for Maps for Nearby Pick-ups

### Importing Data

As above, but throwing away trips with distance values greater than the above defined parameter 'new_distance'.

In [14]:
# load up the workbook and worksheet for working with trips that have nearby pick-up locations
nearby_pickups = pd.read_excel('../data/Nearby Pickups and Dropoffs.xlsx', sheetname='Nearby Pick-ups')

nearby_pickup_coords = {}
for hotel_name in nearby_pickups['Hotel Name'].unique():
    # get the latitude, longitude coordinates of the corresponding pick-up locations for the trips
    satisfying_locations = nearby_pickups.loc[nearby_pickups['Hotel Name'] == hotel_name].loc[nearby_pickups['Distance From Hotel'] <= new_distance]
    nearby_pickup_coords[hotel_name] = np.array(zip(satisfying_locations['Latitude'], satisfying_locations['Longitude'])).T

# delete the 'nearby_pickups' dataframe from memory
del nearby_pickups

In [15]:
for hotel_name in nearby_pickup_coords:
    print 'Number of nearby pick-up taxicab trips (within ' + str(new_distance) + ' feet of hotel ' + hotel_name + '): ' + str(len(nearby_pickup_coords[hotel_name][0]))

Number of nearby pick-up taxicab trips (within 100 feet of hotel Waldorf Astoria New York): 1280
Number of nearby pick-up taxicab trips (within 100 feet of hotel Wyndham New Yorker Hotel): 3038
Number of nearby pick-up taxicab trips (within 100 feet of hotel The Roosevelt Hotel): 3464
Number of nearby pick-up taxicab trips (within 100 feet of hotel Hilton Millenium Hotel): 1307
Number of nearby pick-up taxicab trips (within 100 feet of hotel Marriott New York Marquis): 3007
Number of nearby pick-up taxicab trips (within 100 feet of hotel Hotel Pennsylvania): 20678
Number of nearby pick-up taxicab trips (within 100 feet of hotel Hilton New York Midtown): 1303
Number of nearby pick-up taxicab trips (within 100 feet of hotel Marriott New York Downtown): 1231
Number of nearby pick-up taxicab trips (within 100 feet of hotel Sheraton Hotel New York Times Square): 1198
Number of nearby pick-up taxicab trips (within 100 feet of hotel Grand Hyatt New York): 2832


### Drawing Maps of Corresponding Drop-off Locations (Reduced Distance)

In [16]:
for hotel_name in nearby_pickup_coords:
    # some map parameters
    map_name = hotel_name + '_Jan2016_' + str(new_distance) + 'ft_nearby_pickups_destinations_markers.html'
    filepath = '../img/' + map_name[:-5] + '.png'

    gmap = gmplot.GoogleMapPlotter(np.mean(nearby_pickup_coords[hotel_name][0]), np.mean(nearby_pickup_coords[hotel_name][1]), 13)

    # plot the map
    gmap.scatter(nearby_pickup_coords[hotel_name][0], nearby_pickup_coords[hotel_name][1], color='aquamarine')

    # draw the map
    gmap.draw('../img/' + map_name)

    # display it in the web browser
    webbrowser.open('../img/' + map_name)

## Reducing Distance for Heat Maps for Nearby Drop-offs

### Importing Data

In [17]:
# load up the workbook and worksheet for working with trips that have nearby pick-up locations
nearby_dropoffs = pd.read_excel('../data/Nearby Pickups and Dropoffs.xlsx', sheetname='Nearby Drop-offs')

nearby_dropoff_coords = {}
for hotel_name in nearby_dropoffs['Hotel Name'].unique():
    # get the latitude, longitude coordinates of the corresponding pick-up locations for the trips
    satisfying_locations = nearby_dropoffs.loc[nearby_dropoffs['Hotel Name'] == hotel_name].loc[nearby_dropoffs['Distance From Hotel'] <= new_distance]
    nearby_dropoff_coords[hotel_name] = np.array(zip(satisfying_locations['Latitude'], satisfying_locations['Longitude'])).T

# delete the 'nearby_pickups' dataframe from memory
del nearby_dropoffs

In [18]:
for hotel_name in nearby_dropoff_coords:
    print 'Number of nearby drop-off taxicab trips (within ' + str(new_distance) + ' feet of hotel ' + hotel_name + '): ' + str(len(nearby_dropoff_coords[hotel_name][0]))

Number of nearby drop-off taxicab trips (within 100 feet of hotel Waldorf Astoria New York): 1271
Number of nearby drop-off taxicab trips (within 100 feet of hotel Wyndham New Yorker Hotel): 1890
Number of nearby drop-off taxicab trips (within 100 feet of hotel The Roosevelt Hotel): 2628
Number of nearby drop-off taxicab trips (within 100 feet of hotel Hilton Millenium Hotel): 1023
Number of nearby drop-off taxicab trips (within 100 feet of hotel Marriott New York Marquis): 2660
Number of nearby drop-off taxicab trips (within 100 feet of hotel Hotel Pennsylvania): 12413
Number of nearby drop-off taxicab trips (within 100 feet of hotel Hilton New York Midtown): 1511
Number of nearby drop-off taxicab trips (within 100 feet of hotel Marriott New York Downtown): 1446
Number of nearby drop-off taxicab trips (within 100 feet of hotel Sheraton Hotel New York Times Square): 1214
Number of nearby drop-off taxicab trips (within 100 feet of hotel Grand Hyatt New York): 3111


### Drawing Heat Map of Corresponding Pick-up Locations (Reduced Distance)

In [19]:
for hotel_name in nearby_dropoff_coords:
    # some map parameters
    map_name = hotel_name + '_Jan2016_' + str(new_distance) + 'ft_nearby_dropoffs_starting_points_markers.html'
    filepath = '../img/' + map_name[:-5] + '.png'

    gmap = gmplot.GoogleMapPlotter(np.mean(nearby_dropoff_coords[hotel_name][0]), np.mean(nearby_dropoff_coords[hotel_name][1]), 13)

    # plot the map
    gmap.scatter(nearby_dropoff_coords[hotel_name][0], nearby_dropoff_coords[hotel_name][1], color='aquamarine')

    # draw the map
    gmap.draw('../img/' + map_name)

    # display it in the web browser
    webbrowser.open('../img/' + map_name)